In [1]:
!git clone 'https://github.com/aakashvardhan/s7-in-depth-practice.git'

Cloning into 's7-in-depth-practice'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 33 (delta 10), reused 31 (delta 8), pack-reused 0
Receiving objects: 100% (33/33), 87.67 KiB | 627.00 KiB/s, done.
Resolving deltas: 100% (10/10), done.


In [2]:
import sys
sys.path.append('/content/s7-in-depth-practice')
sys.path.append('/content/s7-in-depth-practice/models')

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model_utils import model_summary, test_model_sanity
from models.model import model1

from utils import train, test

In [4]:
testmodel = model1()
test_model_sanity(testmodel)

100%|██████████| 9912422/9912422 [00:00<00:00, 88328909.87it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 31652911.90it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20513532.73it/s]

...

Epoch 2


Loss: 0.9935 Batch_id=9: 100%|██████████| 10/10 [00:00<00:00, 34.99it/s]


Epoch 3


Loss: 0.1966 Batch_id=9: 100%|██████████| 10/10 [00:00<00:00, 41.52it/s]


Sanity check passed: Model is capable of overfitting to a small subset of the data.


In [5]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [6]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

In [7]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

CUDA Available? True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model1 = model1().to(device)

cuda


In [9]:
optimizer = optim.SGD(model1.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model1, device, train_loader, optimizer, epoch)
    test(model1, device, test_loader)

EPOCH: 0


Loss=0.1169235035777092 Batch_id=468 Accuracy=94.11: 100%|██████████| 469/469 [00:21<00:00, 22.29it/s]



Test set: Average loss: 0.0827, Accuracy: 9747/10000 (97.47%)

EPOCH: 1


Loss=0.03719327226281166 Batch_id=468 Accuracy=98.27: 100%|██████████| 469/469 [00:15<00:00, 30.06it/s]



Test set: Average loss: 0.0454, Accuracy: 9846/10000 (98.46%)

EPOCH: 2


...


Test set: Average loss: 0.0250, Accuracy: 9924/10000 (99.24%)

EPOCH: 13


Loss=0.018597865477204323 Batch_id=468 Accuracy=99.68: 100%|██████████| 469/469 [00:15<00:00, 29.69it/s]



Test set: Average loss: 0.0255, Accuracy: 9930/10000 (99.30%)

EPOCH: 14


Loss=0.005020213313400745 Batch_id=468 Accuracy=99.73: 100%|██████████| 469/469 [00:16<00:00, 28.72it/s]



Test set: Average loss: 0.0264, Accuracy: 9910/10000 (99.10%)

